<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href=https://www.hse.ru/ target="_blank">HSE</a> 2021 onwards</font></small><hr style="margin:0;background-color:silver">

# **[🏆🎞️MovieGenres](https://www.kaggle.com/competitions/oct312022moviegenres/rules)** <small>(See private URL in LMS assignment)</small>

See [**instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running and naming your Colab notebooks.



**(Optioal) CONSENT.** If ok with sharing your Colab for educational purposes, please check the box below with "X".

<mark>[X]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purpose. We understand that sharing is optional and this decision will not affect our grade in any way. 

In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if kaggle.json is stored in Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip -q install -U sentence-transformers > log    # install sentence BERT text encoder
!pip -q install -U --force-reinstall --no-deps kaggle >> log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >>log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json >> log      # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v oct312022moviegenres   # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: oct312022moviegenres
Using competition: oct312022moviegenres
 teamId  teamName                     submissionDate       score    
-------  ---------------------------  -------------------  -------  
9407000  Gleb Kudriashov              2022-11-12 22:21:38  0.92697  
9420438  🐯E-WhereIsTheMarkLebowski    2022-11-11 17:52:52  0.92181  
9440142  🐯D-DwayneScalaJSON-Averin    2022-11-13 10:04:26  0.91952  
9429852  Artem Borisov                2022-11-11 11:38:24  0.91552  
9448666  🐯N-Voronina,Asharin,Almasya  2022-11-12 22:39:59  0.91552  
9419878  🐯Q-RIPBILLY                  2022-11-12 09:28:09  0.91482  
9417635  Artemiy Astashkin            2022-11-11 14:28:15  0.91461  
9412491  Nikita Tyuplyaev             2022-11-12 22:30:55  0.91311  
9379087  🐯I-ABC                       2022-11-12 18:26:36  0.91174  
9421856  Nikita Aksenov               2022-11-12 22:34:30  0.91044  
9417295  islambeg katibov

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, matplotlib.pyplot as plt, plotly, time
from sklearn.model_selection import train_test_split  # the only allowed function from sklearn
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.neighbors import RadiusNeighborsClassifier
from sentence_transformers import SentenceTransformer as SBERT  # to encode multilingual text into numeric vectors

pd.set_option('max_rows', 5, 'max_columns', 100, 'max_colwidth', 30, 'precision', 2)
np.set_printoptions(linewidth=100, precision=2, edgeitems=5, suppress=True)
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

XY = pd.read_csv('MovieGenresXY.csv')
nClass = 20                                         # number of output classes/genres
YCols = XY.columns[-nClass:]                        # 20 output columns - movie genre indicators
# XNumCols = XY.select_dtypes(include=np.number).drop(YCols, axis=1).columns  # numeric column names
# XY[XNumCols] = XY[XNumCols].fillna(0)               # fill numeric columns with zeros
# XY['desc'] = XY.overview + '.' + XY.title + '. ' + XY.original_language + '. ' + XY.keywords + '. ' + XY.tagline
# vX = XY.query('Action!=Action').drop(YCols, axis=1) # test inputs, movie attributes
# tXY = XY.query('Action==Action')                    # training I/O
# tX, tY = tXY.drop(YCols, axis=1), tXY[YCols]        # split into training I/O
# vX, tX = vX.fillna('na'), tX.fillna('na')           # textual fields are filled with NA text
# XY

In [ ]:
# importing some more libraries
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import ast 

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**
Of course, we considered all the textual information in the dataset and cleaned them. Instead of string with dictionary extracted tag words and so on. Also created dummies from companies and countries which produced the film + dummies on the decade of release, but the most benefit came from dummy on film companies creation. Other dummy variables didn't make significant improvement. Also searched for better performing BERT and it helped. Finally, worked with numeric variables, logged some of them and substituted some missing data (nans and zeros) onto something more reasonable. It gave only slight improvement. The performance of everything was checked by cross-val kaggle metric score. 

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:** 
Tried all models, that we can use, but none was better than RidgeClassifier. I used CV to get alpha and selected features based on value of coef (since all vars are normalized, it was somehow reasonable), and feature selection gave a little bit of improvement, but maybe I should be more careful with overfit. So other models didn't work, effectiveness was again checked using cross-validation. 

Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [ ]:
# XY = pd.read_csv('MovieGenresXY.csv')
# nClass = 20                                         # number of output classes/genres
# YCols = XY.columns[-nClass:]                        # 20 output columns - movie genre indicators

# RUNTIME: nan and zeroes in runtime, filling in with avg value!!! BUT CHECK THE PERFORMANCE, MAYBE IT'S WORSE!!!
XY['runtime'] = XY['runtime'].fillna(107)               # fill numeric columns with zeros
XY['runtime'] = XY['runtime'].apply(lambda x: x + 107 if x == 0 else x)
# the thing above gives slight improvement to the cross-val score

# BUDGET: 1000 entries with less than budget, but definitely it should be larger, so replace to more logical vals
  # I am gonna log this vector and fill with random variable from normal distribution to avoid too many single values
# https://stackoverflow.com/questions/47497466/python-fill-na-in-pandas-column-with-random-elements-from-a-list
# https://stackoverflow.com/questions/45416684/python-pandas-replace-multiple-columns-zero-to-nan
mean = np.log(XY[XY['budget'] > 1000]['budget']).mean()
std = np.log(XY[XY['budget'] > 1000]['budget']).std()
XY['incor_budget'] = (XY['budget'] <= 1000).astype('float')
# from less than 1000 to nan
XY['budget_nan'] = XY['budget'].apply(lambda x: np.nan if x < 1000 else x)
XY['log_budget'] = np.log(XY['budget_nan'])
# filling nans with random var: normal mean and std
np.random.seed(42)
XY['log_budget'] = XY['log_budget'].fillna(pd.Series(np.random.normal(mean, std, size=len(XY.index))))
XY = XY.drop('budget', axis = 1)
XY = XY.drop('budget_nan', axis = 1)

# doing the same for REVENUE
mean = np.log(XY[XY['revenue'] > 1000]['revenue']).mean()
std = np.log(XY[XY['revenue'] > 1000]['revenue']).std()
XY['incor_revenue'] = (XY['revenue'] <= 1000).astype('float')
# from less than 1000 to nan
XY['revenue_nan'] = XY['revenue'].apply(lambda x: np.nan if x < 1000 else x)
XY['log_revenue'] = np.log(XY['revenue_nan'])
# filling nans with random var: normal mean and std
np.random.seed(42)
XY['log_revenue'] = XY['log_revenue'].fillna(pd.Series(np.random.normal(mean, std, size=len(XY.index))))
# XY = XY.drop('revenue', axis = 1)
XY = XY.drop('revenue_nan', axis = 1)

# Popularity
XY['log_popularity'] = np.log(1 + XY['popularity'])
XY = XY.drop('popularity', axis = 1)

# Vote_count
XY['log_vote_count'] = np.log(1 + XY['vote_count'])
XY = XY.drop('vote_count', axis = 1)

# VERY SLIGHT IMPROVEMENT OVERALL AFTER NUMERIC COLS TRANSFORMATION


# Text
# for some obs, tagline is Nan, but it's not too big deal, but if it's nan, desc will be also nan!!!
  # To avoid this I first remove nan and then add, so that desc for no tagliners will also exist!!!
# the same for overview, but the scale is not so big (only 3 nans)
XY['tagline'] = XY['tagline'].fillna('')
XY['overview'] = XY['overview'].fillna('')

# KEYWORDS (UPD: there was mistake, it gave improvement)
# https://stackoverflow.com/questions/1894269/how-to-convert-string-representation-of-list-to-a-list
def json_encoder(i):
    li = ast.literal_eval(i)
    try: 
      return ' '.join(list(pd.DataFrame(li)['name']))
    except:
      # when no keywords 
      return ''
XY['keywords_new'] = XY['keywords'].map(json_encoder)

# # ORIGINAL_LANGUAGE: many languages, but en dominant, You can make dummies for langs, non-english: if count in dataset > 10!!!
#   # ALSO NUMBER OF LANGUAGES AS VARIABLE MAY WORK (DUMMIES)
# # (slightly lowers cross-val score!!!)
# dict_or_lang = dict(XY.original_language.value_counts())
# # leaving only the languages with more than 10 observations
# XY['original_language'] = XY['original_language'].apply(lambda x: x if dict_or_lang[x] > 10 else 'other')
# # so that english is dropped in get dummies
# XY['original_language'] = XY['original_language'].apply(lambda x: '0en' if x == 'en' else x)
# XY = pd.get_dummies(XY, drop_first = True, columns = ['original_language'])

# # Status: only 8 status non-released, no need to work with it!!! 

# PRODUCTION COMPANIES: 351 empty, others have company, actually can work with them, but not now I think!! But maybe work with names!!! LEAVE ONLY THOSE DUMMIY COMPANIES, WHICH HAVE >10 FILMS!!!!
  # NUMER OF COMPANIES (DUMMIES) MAY ALSO HELP!!!
# finds all companies for each observation
def prod_companies_encode(i):
    li = ast.literal_eval(i)
    try: 
      return list(pd.DataFrame(li)['name'])
    except:
      # when no dictionary 
      return list()
vec_companies = list(XY['production_companies'].map(prod_companies_encode))
# all companies for all films glued together (leaving repetitions to find overall frequency of certain company!!!)
all_comp = []
comp_counts = []
for i in vec_companies:
  all_comp += i
  comp_counts.append(len(i))
# number of companies for each film!
XY['comp_count'] = comp_counts
# decided to make dummies: more than 0, 1, 2, 5
knots = [0, 1, 2, 5]
for num, i in enumerate(knots): 
    colname = 'comp_count_more_{}'.format(i)
    if num != 3:
        XY[colname] = ((XY['comp_count'] > i) & (XY['comp_count'] <= knots[num + 1])).astype('int')
    else: 
        XY[colname] = (XY['comp_count'] > i).astype('int')
# Frequency of each company overall!!!
comp, count= np.unique(np.array(all_comp), return_counts = True)
df_comps = pd.DataFrame(np.asarray((comp, count)).T)
df_comps[1] = df_comps[1].astype('int')
df_comps = df_comps.sort_values(1, ascending = False)
# IF RUNS TOO LONG, CONSIDER CHANGING THE MIN NUMBER OF FILMS PRODUCED!!!!
df_comps_filt = df_comps[df_comps[1] > 10]
# nice 163 companies: can take all of them and make dummy on companies!!!
comps_to_consider = list(df_comps_filt[0])
# function to catch company in a string of list of dictionaries
def company_dummy(i, company):
    if company in i:
        return 1
    else: 
        return 0
# dummy columns creations
for company in comps_to_consider: 
    colname = 'is_' + company
    XY[colname] = XY['production_companies'].apply(lambda x: company_dummy(x, company))
# Well, 5 companies are not done properly, due to French letters, but it's 5/163 is not big deal in this case
  # we can delete these columns!
comp_cols_avg = XY.iloc[:, -len(comps_to_consider): ].mean(axis = 0)
not_correct_comps = list(comp_cols_avg[comp_cols_avg == 0].index)
XY = XY.drop(not_correct_comps, axis = 1)

# # PRODUCTION COUNTRIES: SIMILAR AS FOR PRODUCTION COUNTRIES
# def prod_countries_encode(i):
#     li = ast.literal_eval(i)
#     try: 
#       return list(pd.DataFrame(li)['iso_3166_1'])
#     except:
#       # when no dictionary 
#       return list()
# list_countries = list(XY['production_countries'].map(prod_countries_encode))
# # all countries for all films glued together (leaving repetitions to find overall frequency of certain countries!!!)
# all_countries = []
# countries_counts = []
# for i in list_countries:
#   all_countries += i
#   countries_counts.append(len(i))
# # number of compa for each film!
# XY['countries_count'] = countries_counts
# # decided to make dummies: more than 0, 1, 2
# knots = [0, 1, 2]
# for num, i in enumerate(knots): 
#     colname = 'countries_count_more_{}'.format(i)
#     if num != 2:
#         XY[colname] = ((XY['countries_count'] > i) & (XY['countries_count'] <= knots[num + 1])).astype('int')
#     else:
#         XY[colname] = (XY['countries_count'] > i).astype('int')
# # Frequency of each countries overall!!!
# countries, count = np.unique(np.array(all_countries), return_counts = True)
# df_countries = pd.DataFrame(np.asarray((countries, count)).T)
# df_countries[1] = df_countries[1].astype('int')
# df_countries = df_countries.sort_values(1, ascending = False)
# # IF RUNS TOO LONG, CONSIDER CHANGING THE MIN NUMBER OF FILMS PRODUCED!!!!
# df_countries_filt = df_countries[df_countries[1] > 10]
# # nice 163 companies: can take all of them and make dummy on companies!!!
# countries_to_consider = list(df_countries_filt[0])
# # function to catch company in a string of list of dictionaries
# def country_dummy(i, country):
#     if country in i:
#         return 1
#     else: 
#         return 0
# # dummy columns creations
# for country in countries_to_consider: 
#     colname = 'is_' + country
#     XY[colname] = XY['production_countries'].apply(lambda x: country_dummy(x, country))
# # Well, 5 companies are not done properly, due to French letters, but it's 5/163 is not big deal in this case
#   # we can delete these columns!
# countries_cols_avg = XY.iloc[:, -len(countries_to_consider): ].mean(axis = 0)
# not_correct_countries = list(countries_cols_avg[countries_cols_avg == 0].index)
# XY = XY.drop(not_correct_countries, axis = 1)

# # SPOKEN LANGUAGE: SAME APPROACH AS PREVIOUS TWO VARIABLES!!!
# def prod_lang_encode(i):
#     li = ast.literal_eval(i)
#     try: 
#       return list(pd.DataFrame(li)['iso_639_1'])
#     except:
#       # when no dictionary 
#       return list()
# list_lang = list(XY['spoken_languages'].map(prod_lang_encode))
# # all countries for all films glued together (leaving repetitions to find overall frequency of certain countries!!!)
# all_lang = []
# lang_counts = []
# for i in list_lang:
#   all_lang += i
#   lang_counts.append(len(i))
# # number of compa for each film!
# XY['lang_count'] = lang_counts
# # decided to make dummies: more than 0, 1, 2
# knots = [0, 1, 2]
# for num, i in enumerate(knots): 
#     colname = 'lang_count_more_{}'.format(i)
#     if num != 2:
#         XY[colname] = ((XY['lang_count'] > i) & (XY['lang_count'] <= knots[num + 1])).astype('int')
#     else: 
#         XY[colname] = (XY['lang_count'] > i).astype('int')
# # Frequency of each countries overall!!!
# lang, count = np.unique(np.array(all_lang), return_counts = True)
# df_lang = pd.DataFrame(np.asarray((lang, count)).T)
# df_lang[1] = df_lang[1].astype('int')
# df_lang = df_lang.sort_values(1, ascending = False)
# # IF RUNS TOO LONG, CONSIDER CHANGING THE MIN NUMBER OF FILMS PRODUCED!!!!
# df_lang_filt = df_lang[df_lang[1] > 10]
# # nice 163 companies: can take all of them and make dummy on companies!!!
# lang_to_consider = list(df_lang_filt[0])
# # function to catch company in a string of list of dictionaries
# def lang_dummy(i, lang):
#     if lang in i:
#         return 1
#     else: 
#         return 0
# # dummy columns creations
# for lang in lang_to_consider: 
#     colname = 'is_' + lang
#     XY[colname] = XY['spoken_languages'].apply(lambda x: lang_dummy(x, lang))
# # Well, 5 companies are not done properly, due to French letters, but it's 5/163 is not big deal in this case
#   # we can delete these columns!
# lang_cols_avg = XY.iloc[:, -len(lang_to_consider): ].mean(axis = 0)
# not_correct_lang = list(lang_cols_avg[lang_cols_avg == 0].index)
# XY = XY.drop(not_correct_lang, axis = 1)

# # RELEASE DATE - may create dummies based on decades!!
# # https://stackoverflow.com/questions/17764619/pandas-dataframe-group-year-index-by-decade
# # spoken_languages: based on codes, make dummies whether certain code is in dictionary!!!
#   # NUMBER OF LANGUAGES(DUMMIES) MAY BE HELPFUL!
# # DELETE ALL CREATED DUMMIES, IF NUMBER OF '1' IS LESS THAN 10 OR SOME OTHER BENCHMARK!!!
# # only one film with NA, which was released in 2015!
# XY['release_date'] = XY['release_date'].fillna('2015-01-01')
# XY['release_date'] = pd.to_datetime(XY.release_date)
# XY['decade_release'] = (XY.release_date.dt.year//10)*10
# XY['decade_release_bef_1950'] = (XY['decade_release'] < 1950).astype('int')
# XY['decade_release_1950_1960'] = ((XY['decade_release'] >= 1950) & (XY['decade_release'] < 1970)).astype('int')
# XY['decade_release_70s'] = (XY['decade_release'] == 1970).astype('int')
# XY['decade_release_80s'] = (XY['decade_release'] == 1980).astype('int')
# XY['decade_release_90s'] = (XY['decade_release'] == 1990).astype('int')
# XY['decade_release_2000s'] = (XY['decade_release'] == 2000).astype('int')

# ok, the only numeric columns (with dummies from text variables)
XNumCols = XY.select_dtypes(include=np.number).drop(YCols, axis=1).columns  # numeric column names

XY['desc'] = XY.overview + '.' + XY.title + '. ' + XY.keywords_new + '. ' + XY.tagline
vX = XY.query('Action!=Action').drop(YCols, axis=1) # test inputs, movie attributes
tXY = XY.query('Action==Action')                    # training I/O
tX, tY = tXY.drop(YCols, axis=1), tXY[YCols]        # split into training I/O
vX, tX = vX.fillna('na'), tX.fillna('na')           # textual fields are filled with NA text
XY

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:125: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


,homepage,keywords,original_language,original_title,overview,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,incor_budget,log_budget,incor_revenue,log_revenue,log_popularity,log_vote_count,keywords_new,comp_count,comp_count_more_0,comp_count_more_1,comp_count_more_2,comp_count_more_5,is_Warner Bros.,is_Universal Pictures,is_Paramount Pictures,...,is_Atlas Entertainment,is_Mel's Cite du Cinema,is_Paramount Vantage,is_Robert Simonds Productions,is_Marvel Studios,is_Channel Four Films,is_2929 Productions,is_Nu Image Films,is_Saturn Films,is_Scion Films,is_RatPac-Dune Entertainment,"is_Kennedy/Marshall Company, The",is_BenderSpink,is_Tim Burton Productions,is_Sunswept Entertainment,is_3 Arts Entertainment,is_Goldcrest Pictures,is_Bad Hat Harry Productions,is_LStar Capital,is_Moving Picture Company (MPC),is_Gold Circle Films,is_CBS Films,is_Media Rights Capital,is_Worldview Entertainment,is_Icon Entertainment International,is_The Kennedy/Marshall Company,is_Dentsu,is_Impact Pictures,is_Cinergi Pictures Entertainment,is_Mandate Pictures,is_Emmett/Furla Films,is_Endgame Entertainment,is_21 Laps Entertainment,is_Davis-Films,is_DC Entertainment,is_Sony Pictures Animation,is_Destination Films,is_British Broadcasting Corporation (BBC),is_Good Machine,is_Marc Platt Productions,is_Paramount Animation,is_Twentieth Century Fox Animation,is_Fine Line Features,is_Miramax,is_Mandeville Films,is_WingNut Films,is_IM Global,is_DiNovi Pictures,is_Alliance Atlantis Communications,desc
0,NaN,"[{""id"": 907, ""name"": ""japa...",en,Hachi: A Dog's Tale,A drama based on the true ...,"[{""name"": ""Grand Army Ente...","[{""iso_3166_1"": ""GB"", ""nam...",2009-06-13,47801389,93.0,"[{""iso_639_1"": ""en"", ""name...",Released,"A true story of faith, dev...",Hachi: A Dog's Tale,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.59,0.0,17.68,3.79,7.45,japanese loyalty human ani...,6,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,A drama based on the true ...
1,NaN,"[{""id"": 520, ""name"": ""chic...",en,The Sting,Set in the 1930's this int...,"[{""name"": ""Universal Pictu...","[{""iso_3166_1"": ""US"", ""nam...",1973-12-25,159616327,129.0,"[{""iso_639_1"": ""en"", ""name...",Released,...all it takes is a littl...,The Sting,7.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.52,0.0,18.89,3.38,6.43,chicago bet horse race rep...,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Set in the 1930's this int...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4801,http://www.biglebowskiblur...,"[{""id"": 418, ""name"": ""whit...",en,The Big Lebowski,"Jeffrey ""The Dude"" Lebowsk...","[{""name"": ""Gramercy Pictur...","[{""iso_3166_1"": ""GB"", ""nam...",1998-03-06,46189568,117.0,"[{""iso_639_1"": ""en"", ""name...",Released,Times like these call for ...,The Big Lebowski,7.8,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.52,0.0,17.65,3.92,7.98,white russian dude bowling...,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Jeffrey ""The Dude"" Lebowsk..."
4802,NaN,"[{""id"": 4118, ""name"": ""bal...",en,Save the Last Dance,A white midwestern girl mo...,"[{""name

### Checking numeric columns for outliers (all of the final changes will be done above!!!)

In [ ]:
# BUDGET: 1000 entries with less than budget, but definitely it should be larger, so replace to more logical vals
  # I am gonna log this vector and fill with random variable from normal distribution to avoid too many single values
# https://stackoverflow.com/questions/47497466/python-fill-na-in-pandas-column-with-random-elements-from-a-list
# https://stackoverflow.com/questions/45416684/python-pandas-replace-multiple-columns-zero-to-nan

# mean = np.log(XY[XY['budget'] > 1000]['budget']).mean()
# std = np.log(XY[XY['budget'] > 1000]['budget']).std()
# # from less than 1000 to nan
# XY['budget'] = XY['budget'].apply(lambda x: np.nan if x < 1000 else x)
# XY['log_budget'] = np.log(XY['budget'])
# # filling nans with random var: normal mean and std
# np.random.seed(42)
# XY['log_budget'] = XY['log_budget'].fillna(pd.Series(np.random.normal(mean, std, size=len(XY.index))))
# XY = XY.drop('budget', axis = 1)

# sns.kdeplot(XY['log_budget'])

# POPULARITY: No outliers in my opinion, can just log
# sns.histplot(np.log(1 + XY['popularity']))

# also maybe makes sense to log VOTE_COUNT!

# doing the same for revenue as for budget!!!!
# mean = np.log(XY[XY['revenue'] > 1000]['revenue']).mean()
# std = np.log(XY[XY['revenue'] > 1000]['revenue']).std()
# # from less than 1000 to nan
# XY['revenue'] = XY['revenue'].apply(lambda x: np.nan if x < 1000 else x)
# XY['log_budget'] = np.log(XY['revenue'])
# # filling nans with random var: normal mean and std
# np.random.seed(42)
# XY['log_revenue'] = XY['log_revenue'].fillna(pd.Series(np.random.normal(mean, std, size=len(XY.index))))
# XY = XY.drop('revenue', axis = 1)
# sns.histplot(XY['revenue'])

### Working with textual columns

In [ ]:
# ORIGINAL LANGUAGE would be translated to dummies
# For OVERVIEW AND TAGLINE AND TITLE it should be fine (removed nans onto '' so that single Nans will not destroy desc col)
# As for KEYWORDS, I need to extract them, they are in messy format!!!
# XY['desc'] = XY.overview + '.' + XY.title +  '. ' + XY.keywords + '. ' + XY.tagline
# XY.overview[0]

In [ ]:
# # https://stackoverflow.com/questions/1894269/how-to-convert-string-representation-of-list-to-a-list
# def json_encoder(i):
#     li = ast.literal_eval(i)
#     return ' '.join(list(pd.DataFrame(li)['name']))

# XY['keywords_new'] = XY['keywords'].map(json_encoder)

In [ ]:
# # ORIGINAL_LANGUAGE: many languages, but en dominant, You can make dummies for langs, non-english: if count in dataset > 10!!!
#   # ALSO NUMBER OF LANGUAGES AS VARIABLE MAY WORK (DUMMIES)
# dict_or_lang = dict(XY.original_language.value_counts())
# # leaving only the languages with more than 10 observations
# XY['original_language'] = XY['original_language'].apply(lambda x: x if dict_or_lang[x] > 10 else 'other')
# # so that english is dropped in get dummies
# XY['original_language'] = XY['original_language'].apply(lambda x: '0en' if x == 'en' else x)
# pd.DataFrame(XY['original_language'].value_counts())
# XY = pd.get_dummies(XY, drop_first = True, columns = ['original_language'])


In [ ]:
# Status: only 8 status non-released, no need to work with it!!! 

# PRODUCTION COMPANIES: 351 empty, others have company, actually can work with them, but not now I think!! But maybe work with names!!! LEAVE ONLY THOSE DUMMIY COMPANIES, WHICH HAVE >10 FILMS!!!!
  # NUMER OF COMPANIES (DUMMIES) MAY ALSO HELP!!!
# finds all companies for each observation
# def prod_companies_encode(i):
#     li = ast.literal_eval(i)
#     try: 
#       return list(pd.DataFrame(li)['name'])
#     except:
#       # when no dictionary 
#       return list()
# vec_companies = list(XY['production_companies'].map(prod_companies_encode))
# # all companies for all films glued together (leaving repetitions to find overall frequency of certain company!!!)
# all_comp = []
# comp_counts = []
# for i in vec_companies:
#   all_comp += i
#   comp_counts.append(len(i))
# # number of companies for each film!
# XY['comp_count'] = comp_counts
# # decided to make dummies: more than 0, 1, 2, 5
# for i in [0, 1, 2, 5]: 
#     colname = 'comp_count_more_{}'.format(i)
#     XY[colname] = XY['comp_count'] > i

# # Number of each company overall!!!
# comp, count= np.unique(np.array(all_comp), return_counts = True)
# df_comps = pd.DataFrame(np.asarray((comp, count)).T)
# df_comps[1] = df_comps[1].astype('int')
# df_comps = df_comps.sort_values(1, ascending = False)
# # IF RUNS TOO LONG, CONSIDER CHANGING THE MIN NUMBER OF FILMS PRODUCED!!!!
# df_comps_filt = df_comps[df_comps[1] > 10]
# # nice 163 companies: can take all of them and make dummy on companies!!!
# comps_to_consider = list(df_comps_filt[0])
# # function to catch company in a string of list of dictionaries
# def company_dummy(i, company):
#     if company in i:
#         return 1
#     else: 
#         return 0

# # dummy columns creations
# for company in comps_to_consider: 
#     colname = 'is_' + company
#     XY[colname] = XY['production_companies'].apply(lambda x: company_dummy(x, company))
# # Well, 5 companies are not done properly, due to French letters, but it's 5/163 is not big deal in this case
#   # we can delete these columns!
# comp_cols_avg = XY.iloc[:, -len(comps_to_consider): ].mean(axis = 0)
# not_correct_comps = list(comp_cols_avg[comp_cols_avg == 0].index)
# XY = XY.drop(not_correct_comps, axis = 1)

In [ ]:
# # PRODUCTION COUNTRIES!!!
# def prod_countries_encode(i):
#     li = ast.literal_eval(i)
#     try: 
#       return list(pd.DataFrame(li)['iso_3166_1'])
#     except:
#       # when no dictionary 
#       return list()
# list_countries = list(XY['production_countries'].map(prod_countries_encode))
# # all countries for all films glued together (leaving repetitions to find overall frequency of certain countries!!!)
# all_countries = []
# countries_counts = []
# for i in list_countries:
#   all_countries += i
#   countries_counts.append(len(i))
# # number of compa for each film!
# XY['countries_count'] = countries_counts
# # decided to make dummies: more than 0, 1, 2
# for i in [0, 1, 2]: 
#     colname = 'countries_count_more_{}'.format(i)
#     XY[colname] = (XY['countries_count'] > i).astype('int')
# # Frequency of each countries overall!!!
# countries, count = np.unique(np.array(all_countries), return_counts = True)
# df_countries = pd.DataFrame(np.asarray((countries, count)).T)
# df_countries[1] = df_countries[1].astype('int')
# df_countries = df_countries.sort_values(1, ascending = False)
# # IF RUNS TOO LONG, CONSIDER CHANGING THE MIN NUMBER OF FILMS PRODUCED!!!!
# df_countries_filt = df_countries[df_countries[1] > 10]
# # nice 163 companies: can take all of them and make dummy on companies!!!
# countries_to_consider = list(df_countries_filt[0])
# # function to catch company in a string of list of dictionaries
# def country_dummy(i, country):
#     if country in i:
#         return 1
#     else: 
#         return 0
# # dummy columns creations
# for country in countries_to_consider: 
#     colname = 'is_' + country
#     XY[colname] = XY['production_countries'].apply(lambda x: country_dummy(x, country))
# # Well, 5 companies are not done properly, due to French letters, but it's 5/163 is not big deal in this case
#   # we can delete these columns!
# countries_cols_avg = XY.iloc[:, -len(countries_to_consider): ].mean(axis = 0)
# not_correct_countries = list(countries_cols_avg[countries_cols_avg == 0].index)
# XY = XY.drop(not_correct_countries, axis = 1)

In [ ]:
# # SPOKEN LANGUAGE: SAME APPROACH AS PREVIOUS TWO VARIABLES!!!
# def prod_lang_encode(i):
#     li = ast.literal_eval(i)
#     try: 
#       return list(pd.DataFrame(li)['iso_639_1'])
#     except:
#       # when no dictionary 
#       return list()
# list_lang = list(XY['spoken_languages'].map(prod_lang_encode))
# # all countries for all films glued together (leaving repetitions to find overall frequency of certain countries!!!)
# all_lang = []
# lang_counts = []
# for i in list_lang:
#   all_lang += i
#   lang_counts.append(len(i))
# # number of compa for each film!
# XY['lang_count'] = lang_counts
# # decided to make dummies: more than 0, 1, 2
# for i in [0, 1, 2]: 
#     colname = 'lang_count_more_{}'.format(i)
#     XY[colname] = (XY['lang_count'] > i).astype('int')
# # Frequency of each countries overall!!!
# lang, count = np.unique(np.array(all_lang), return_counts = True)
# df_lang = pd.DataFrame(np.asarray((lang, count)).T)
# df_lang[1] = df_lang[1].astype('int')
# df_lang = df_lang.sort_values(1, ascending = False)
# # IF RUNS TOO LONG, CONSIDER CHANGING THE MIN NUMBER OF FILMS PRODUCED!!!!
# df_lang_filt = df_lang[df_lang[1] > 10]
# # nice 163 companies: can take all of them and make dummy on companies!!!
# lang_to_consider = list(df_lang_filt[0])
# # function to catch company in a string of list of dictionaries
# def lang_dummy(i, lang):
#     if lang in i:
#         return 1
#     else: 
#         return 0
# # dummy columns creations
# for lang in lang_to_consider: 
#     colname = 'is_' + lang
#     XY[colname] = XY['spoken_languages'].apply(lambda x: lang_dummy(x, lang))
# # Well, 5 companies are not done properly, due to French letters, but it's 5/163 is not big deal in this case
#   # we can delete these columns!
# lang_cols_avg = XY.iloc[:, -len(lang_to_consider): ].mean(axis = 0)
# not_correct_lang = list(lang_cols_avg[lang_cols_avg == 0].index)
# XY = XY.drop(not_correct_lang, axis = 1)

In [ ]:
# RELEASE DATE - may create dummies based on decades!!
# https://stackoverflow.com/questions/17764619/pandas-dataframe-group-year-index-by-decade
# spoken_languages: based on codes, make dummies whether certain code is in dictionary!!!
  # NUMBER OF LANGUAGES(DUMMIES) MAY BE HELPFUL!

# DELETE ALL CREATED DUMMIES, IF NUMBER OF '1' IS LESS THAN 10 OR SOME OTHER BENCHMARK!!!

# only one film with NA, which was released in 2015!
# XY['release_date'] = XY['release_date'].fillna('2015-01-01')
# XY['release_date'] = pd.to_datetime(XY.release_date)
# XY['decade_release'] = (XY.release_date.dt.year//10)*10
# XY['decade_release_bef_1950'] = XY['decade_release'] < 1950
# XY['decade_release_1950_1960'] = (XY['decade_release'] >= 1950) & (XY['decade_release'] < 1970)
# XY['decade_release_70s'] = XY['decade_release'] == 1970
# XY['decade_release_80s'] = XY['decade_release'] == 1980
# XY['decade_release_90s'] = XY['decade_release'] == 1990
# XY['decade_release_2000s'] = XY['decade_release'] == 2000

### Embedding part!!

[SBERT](https://www.sbert.net) generates 384-dimensional text embedding vectors for each movie's description. See [more models](https://www.sbert.net/docs/pretrained_models.html). Use GPU runtime in Colab for 10-100x speed up.

In [ ]:
# Initial: paraphrase-MiniLM-L6-v2

# new embedding
%time sbert = SBERT('all-mpnet-base-v2')  # load SBERT embedings model to encode textual fields

CPU times: user 1.04 s, sys: 514 ms, total: 1.56 s
Wall time: 1.73 s


In [ ]:
%%time 
train_encode = sbert.encode(tX.desc.tolist())
val_encode = sbert.encode(vX.desc)

CPU times: user 27.2 s, sys: 204 ms, total: 27.4 s
Wall time: 32.5 s


In [ ]:
# actual embedding
tXEmb = np.c_[tX[XNumCols].values, train_encode] # Set GPU runtime in Colab for 10-100x speed up
vXEmb = np.c_[vX[XNumCols].values, val_encode]
print(f'Train embedding matrix size:', tXEmb.shape)
pd.DataFrame(tXEmb[:3,:20], index=tXY.title[:3]).style.background_gradient(cmap='coolwarm')  # show movie description and a few of its embedding features

Train embedding matrix size: (2401, 940)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
title,,,,,,,,,,,,,,,,,,,,
The Midnight Meat Train,3533227.00,98.00,6.00,0.00,16.52,0.00,15.08,3.04,5.67,4.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
London to Brighton,0.00,85.00,6.50,1.00,16.30,1.00,16.97,1.40,3.56,3.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
The Wash,10229331.00,93.00,5.30,0.00,15.76,0.00,16.14,1.24,3.30,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
vXEmb.shape

(2402, 940)

In [ ]:
# m = RidgeClassifier(random_state=0)  # multi-label model
# %time m.fit(tXEmb, tY)               # fitting to training I/O

In [ ]:
# # cross-val score based on metric proposed by Gleb Kudryashov: https://discourse.hsemlp.ru/t/moviegenres-evaluation-metric/371/3
# # this should be the metric used by Kaggle
# n_splits = 5
# kf = KFold(n_splits = n_splits)
# # 20 classes
# score_matrix = np.zeros ((n_splits, 20))
# best_alphas = []

# for j, (train_index, test_index) in enumerate(kf.split(tXEmb)):

#     X_train, X_test = tXEmb[train_index], tXEmb[test_index]
#     y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

#     # as at this moment not normalized slightly better (770), but better normalized for further
#     # only scaling, non-dummy variables!!! (DIDN'T WORK!!!)
#     scaler = StandardScaler()
#     X_train[:, :] = scaler.fit_transform(X_train[:, :])
#     X_test[:, :] = scaler.transform(X_test[:, :])

#     # slightly better and more stable with scaler!
#     m = RidgeClassifierCV(alphas = [1, 2, 5, 10, 20, 50, 100, 200, 500, 750, 1000, 1250, 1500, 1750, 2000, 2250], cv = 5)
#     m = RidgeClassifier(alpha = 1500, random_state = 0)
#     m.fit(X_train, y_train);
    
#     #best_alphas.append(m.alpha_)

#     pred = m.predict(X_test)

#     for i in range(20):
#       score_matrix[j, i] = f1_score(y_test.iloc[:, i], pred[:, i], average = 'micro')

# print('\n')
# print(score_matrix.mean())
# print(list(np.round(score_matrix.mean(axis = 1) * 100, 2)))
# print(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))
# print(best_alphas)

## Well, non-normalized better but shrinkage is less in non-normalized, so I will try to select variables from normalized with low shrinkage and re-run normalized with selected variables at the low shrinkage

# Feature selection: choosing ones with highest values of coefficient (sine normalized features)

In [ ]:
# which classes we want to improve
# perf_classes = np.array(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))
# np.where(perf_classes < 9.5)

In [ ]:
# # cross-val score based on metric proposed by Gleb Kudryashov: https://discourse.hsemlp.ru/t/moviegenres-evaluation-metric/371/3
# # this should be the metric used by Kaggle
# n_splits = 5
# kf = KFold(n_splits = n_splits)
# # 20 classes
# score_matrix = np.zeros((n_splits, 20))
# best_alphas = []

# coefs_matrix = np.zeros((5, 20, tXEmb.shape[1]))

# for j, (train_index, test_index) in enumerate(kf.split(tXEmb)):

#     X_train, X_test = tXEmb[train_index], tXEmb[test_index]
#     y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

#     # as at this moment not normalized slightly better (770), but better normalized for further
#     # only scaling, non-dummy variables!!! (DIDN'T WORK!!!)
#     scaler = StandardScaler()
#     X_train[:, :] = scaler.fit_transform(X_train[:, :])
#     X_test[:, :] = scaler.transform(X_test[:, :])

#     # slightly better and more stable with scaler!
#     m = RidgeClassifierCV(alphas = [10, 20, 50, 100], cv = 5)
#     m.fit(X_train, y_train);
    
#     best_alphas.append(m.alpha_)

#     coefs_matrix[j] = m.coef_

#     pred = m.predict(X_test)

#     for i in range(20):
#       score_matrix[j, i] = f1_score(y_test.iloc[:, i], pred[:, i], average = 'micro')

# print('\n')
# print(score_matrix.mean())
# print(list(np.round(score_matrix.mean(axis = 1) * 100, 2)))
# print(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))
# print(best_alphas)

In [ ]:
# only relevant targets!!!
# abs_avg_coefs = np.abs(coefs_matrix[:, [ 0,  1,  3,  4,  6,  7,  8, 11, 13, 14, 15, 17], :].mean(axis = 0)).sum(axis = 0)

In [ ]:
# so choose only top k with highest values, vary alpha in the cell above!!!!
# abs_avg_coefs

In [ ]:
# selected_vars = list(np.where(abs_avg_coefs > 0.17)[0])

In [ ]:
# tXEmb[train_index][:, selected_vars].shape

## Best model with selected features

In [ ]:
# # cross-val score based on metric proposed by Gleb Kudryashov: https://discourse.hsemlp.ru/t/moviegenres-evaluation-metric/371/3
# # this should be the metric used by Kaggle
# n_splits = 5
# kf = KFold(n_splits = n_splits)
# # 20 classes
# score_matrix = np.zeros((n_splits, 20))
# best_alphas = []

# coefs_matrix = np.zeros((5, 20, tXEmb[:, selected_vars].shape[1]))

# for j, (train_index, test_index) in enumerate(kf.split(tXEmb)):

#     # selected vars!!!!
#     X_train, X_test = tXEmb[train_index][:, selected_vars], tXEmb[test_index][:, selected_vars]
#     y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

#     # as at this moment not normalized slightly better (770), but better normalized for further
#     # only scaling, non-dummy variables!!! (DIDN'T WORK!!!)
#     scaler = StandardScaler()
#     X_train[:, :] = scaler.fit_transform(X_train[:, :])
#     X_test[:, :] = scaler.transform(X_test[:, :])

#     # slightly better and more stable with scaler!
#     m = RidgeClassifierCV(alphas = [20, 50, 100, 200, 500, 1000, 1500, 2000], cv = 5)
#     m = RidgeClassifier(alpha = 500, random_state = 0)
#     m.fit(X_train, y_train);
    
#     #best_alphas.append(m.alpha_)

#     coefs_matrix[j] = m.coef_

#     pred = m.predict(X_test)

#     for i in range(20):
#       score_matrix[j, i] = f1_score(y_test.iloc[:, i], pred[:, i], average = 'micro')

# print('\n')
# print(score_matrix.mean())
# print(list(np.round(score_matrix.mean(axis = 1) * 100, 2)))
# print(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))
# print(best_alphas)

## Other models (no success, but LGBM was quite close)

In [ ]:
# cross-val score based on metric proposed by Gleb Kudryashov: https://discourse.hsemlp.ru/t/moviegenres-evaluation-metric/371/3
# this should be the metric used by Kaggle

# n_splits = 5
# kf = KFold(n_splits = n_splits)
# # 20 classes
# score_matrix = np.zeros ((n_splits, 20))

# for j, (train_index, test_index) in enumerate(kf.split(tXEmb)):

#     X_train, X_test = tXEmb[train_index][:, selected_vars], tXEmb[test_index][:, selected_vars]
#     y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

#     for i in range(20):
#       m = LGBMClassifier()
#       m.fit(X_train, y_train.iloc[:, i]);

#       pred = m.predict(X_test)
#       score_matrix[j, i] = f1_score(y_test.iloc[:, i], pred, average = 'micro')

# print('\n')

# print(score_matrix.mean())
# print(list(np.round(score_matrix.mean(axis = 1) * 100, 2)))
# print(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))

In [ ]:
# # best model scores: old bert
# [8.42, 8.67, 9.67, 7.94, 8.75, 9.78, 7.58, 9.23, 9.23, 9.92, 9.55, 9.18, 9.63, 9.2, 8.58, 9.28, 9.98, 8.01, 9.65, 9.8]
# # best model score: new bert
# [8.57, 8.74, 9.68, 8.07, 8.84, 9.78, 7.28, 9.35, 9.26, 9.92, 9.61, 9.41, 9.66, 9.22, 8.61, 9.41, 9.98, 8.0, 9.7, 9.8]

In [ ]:
#KNN: Unsuccessful!!!
# # cross-val score based on metric proposed by Gleb Kudryashov: https://discourse.hsemlp.ru/t/moviegenres-evaluation-metric/371/3
# # this should be the metric used by Kaggle
# n_splits = 5
# kf = KFold(n_splits = n_splits)
# # 20 classes
# score_matrix = np.zeros ((n_splits, 20))

# for j, (train_index, test_index) in enumerate(kf.split(tXEmb)):

#     X_train, X_test = tXEmb[train_index][:, selected_vars], tXEmb[test_index][:, selected_vars]
#     y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

#     m = KNeighborsClassifier(n_neighbors = 10)
#     m.fit(X_train, y_train);

#     pred = m.predict_proba(X_test)
#     pred_proba = np.zeros((X_test.shape[0], 20))
#     for num, _ in enumerate(pred):
#         pred_proba[:, num] = pred[num][:, 1]

#     pred_final = np.int_(pred_proba > 0.3)

#     for i in range(20):
#       score_matrix[j, i] = f1_score(y_test.iloc[:, i], pred_final[:, i], average = 'micro')

# print('\n')

# print(score_matrix.mean())
# print(list(np.round(score_matrix.mean(axis = 1) * 100, 2)))
# print(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))

In [ ]:
# Random Forest no success

# # cross-val score based on metric proposed by Gleb Kudryashov: https://discourse.hsemlp.ru/t/moviegenres-evaluation-metric/371/3
# # this should be the metric used by Kaggle
# n_splits = 5
# kf = KFold(n_splits = n_splits)
# # 20 classes
# score_matrix = np.zeros ((n_splits, 20))

# for j, (train_index, test_index) in enumerate(kf.split(tXEmb)):

#     X_train, X_test = tXEmb[train_index][:, selected_vars], tXEmb[test_index][:, selected_vars]
#     y_train, y_test = tY.iloc[train_index], tY.iloc[test_index]

#     m = RandomForestClassifier(n_estimators = 100)
#     m.fit(X_train, y_train);

#     pred = m.predict_proba(X_test)
#     pred_proba = np.zeros((X_test.shape[0], 20))
#     for num, _ in enumerate(pred):
#         pred_proba[:, num] = pred[num][:, 1]

#     pred_final = np.int_(pred_proba > 0.45)

#     for i in range(20):
#       score_matrix[j, i] = f1_score(y_test.iloc[:, i], pred_final[:, i], average = 'micro')

# print('\n')

# print(score_matrix.mean())
# print(list(np.round(score_matrix.mean(axis = 1) * 100, 2)))
# print(list(np.round(score_matrix.mean(axis = 0) * 10, 2)))

# Final models and making predictions!

In [ ]:
# features selected previously!!!!
selected_vars = [0, 1, 2, 4, 6, 7, 8, 11, 12, 15, 16, 21, 22, 25, 30, 34, 38, 47, 58, 60, 61, 62, 63, 77, 78, 79, 85, 101, 104, 106, 109, 129, 131, 158,
 173, 174, 175, 181, 182, 185, 186, 187, 188, 189, 191, 194, 196, 197, 198, 200, 201, 202, 203, 206, 207, 208, 209, 210, 211, 212, 213, 216, 218, 219, 221, 222, 223, 225, 226, 230,  
 231,  234,  236,  237,  238,  239,  240,  242,  243,  244,  245,  248,  249,  250,  251,  252,  253,  254,  255,  256,  257,  258,  260,  261,  262,  266,  267,  269,  270,  272,  275,  276,  277,  280,  283,  284,  288,  290,  291,  293,  295,  298,  299,  300,  301,  302,  304,  305,  306,  308,  310,
 311, 313, 315, 316, 317, 318, 320, 326, 327, 328, 329, 330, 333, 334, 335, 336, 339, 340, 342, 344, 345, 347, 348, 349, 350, 352, 353, 354, 355, 356, 358, 360, 363, 364, 367, 370, 371, 372, 373, 374, 375, 379, 382, 383, 384, 385, 386, 388, 389, 390, 392, 393, 394, 395,
 397, 398, 399, 400, 402, 403, 405, 406, 407, 408, 411, 412, 413, 415, 418, 420, 422, 425, 426, 430, 432, 434, 435, 436, 437, 438, 439, 441, 442, 446, 448, 449, 450, 451, 452, 454, 455, 456, 458, 460, 462, 463, 464, 469, 470, 471, 472, 474, 476, 478, 481, 482, 483,
 488, 489, 490, 493, 494, 495, 497, 499, 500, 501, 502, 503, 504, 505, 507, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 521, 523, 525, 529, 530, 531, 532, 533, 535, 536, 537, 538, 540, 541, 542, 543, 546, 547, 548, 552, 554, 557, 558, 560, 561, 565, 566, 567, 569,
 570, 571, 573, 574, 575, 576, 577, 579, 580, 581, 582, 586, 587, 588, 590, 591, 593, 594, 595, 597, 598, 599, 600, 603, 604, 605, 607, 608, 609, 615, 616, 618, 620, 622, 625, 626, 627, 628, 629, 630, 634, 635, 637, 638, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 651, 655, 656, 657,
 658, 659, 660, 661, 662, 663, 664, 665, 667, 668, 669, 670, 671, 672, 674, 675, 676, 677, 678, 679, 681, 683, 684, 686, 688, 689, 690, 691, 692, 693, 696, 700, 701, 704, 705, 710, 711, 712, 713, 715, 716, 717, 718, 719, 721, 722, 723, 724, 727, 730, 732, 734, 735, 736, 738, 739, 740, 743, 746, 749,
 750, 753, 754, 755, 756, 757, 758, 759, 760, 761, 762, 763, 764, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 779, 780, 781, 783, 784, 786, 788, 789, 790, 793, 794, 795, 796, 799, 800, 801, 804, 805, 806, 807, 808, 809, 811, 812, 813, 
 814, 815, 816, 818, 820, 821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 833, 834, 835, 836, 837, 839, 840, 841, 842, 843, 845, 846, 847, 848, 849, 850, 851, 852, 853, 854, 855, 857, 858, 859, 860, 862, 865, 866, 867, 869, 870, 872, 873, 874, 877, 878, 880, 881, 882, 883, 885, 886, 887, 891,
 892, 893, 897, 898, 899, 900, 904, 907, 909, 910, 911, 914, 915, 916, 917, 918, 919, 920, 921, 922, 924, 925, 926, 927, 929, 930, 931, 932, 934, 935, 936, 938]

In [ ]:
# current final model
X_train = tXEmb[:, selected_vars]
y_train = tY
X_test = vXEmb[:, selected_vars]

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# for model with selected features best alpha is 650!!!!
#m = RidgeClassifierCV(alphas = [5, 10, 15, 20, 30, 50, 75, 100, 200, 350, 500, 650, 750, 900, 1000, 1250, 1500], cv = 5)
# using alpha 650
m = RidgeClassifier(alpha = 650, random_state = 0)
m.fit(X_train, y_train)

#m.alpha_

RidgeClassifier(alpha=650, random_state=0)

In [ ]:
pY = pd.DataFrame(m.predict(X_test), index=range(len(vX)), columns=YCols)  # Don't shuffle observations. Keep original order with index 0,1,2,...
pd.DataFrame(pY.values[:10,:], index=vX.title[:10], columns=YCols).style.background_gradient(cmap='coolwarm', axis=1)

# UNCOMMENT THIS LINE TO SAVE FILE!!!
#pY.reset_index().rename(columns={'index':'id'}).to_csv('All_feature_extraction_feature_selection_new_bert_normalized1_less_features1.csv', index=False)  

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,Foreign,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
title,,,,,,,,,,,,,,,,,,,,
Hachi: A Dog's Tale,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
The Sting,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
The Book of Life,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
"Dude, Where’s My Car?",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bolt,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Saw IV,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
Hall Pass,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
The Ladies Man,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Jingle All the Way,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


# **References:**

1. Remember to cite your sources here as well! At the least, your textbook should be cited. Google Scholar allows you to effortlessly copy/paste an APA citation format for books and publications. Also cite StackOverflow, package documentation, and other meaningful internet resources to help your peers learn from these (and to avoid plagiarism claims).

<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 42 sec


## around 63 seconds on GPU. Note that this code should generate 0.91885 public score!!!

# Note that currently this notebook is not connected to GPU. Switch to GPU, so that the notebook will run faster!

# 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features for the original numeric features. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in suitable decision boundaries between pairs of classes (for example, linear boundary for linear models).
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries. Is there a way to separate mixed points near the boundary by, perhaps, introducing additional dimensions or interactions?
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. Clean up textual fields to remove uninformative text. For example, you can use [`json`](https://docs.python.org/3/library/json.html) and [`re`](https://docs.python.org/3/library/re.html) to retrieve just keywords from the lists of [JSON](https://en.wikipedia.org/wiki/JSON) keywords. This may speed up embeddings and lower noise in output coefficients.
   1. Example: `'[{"id": 907, "name": "japanese"}, ...'` $\to$ `'japanese, loyalty, friendship, ...'`
1. Fill input `NA`'s with more suitable statistic (for example, column or group mean or median)
1. Replaces extreme numeric values (such as zeros) with some statistic (such as mean/median) or a modeled value or `NA` (if the predictive model can handle `NA` inputs)
1. Consider embedding other textual fields, if they appear to relate to genres. For example, some countries or companies may focus on documentaries or on action films
1. Try to find a more [suitable SBERT embedding](https://www.sbert.net/docs/pretrained_models.html
1. Consider embedding "important" (for prediction) textual fields separately and concatenating or summing their vectors.
) (small, fast, trained on related text)
1. Learn about [TMDB](https://www.themoviedb.org/) dataset and [related models](https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q=tmdb+machine+learning+model&btnG=).
